In [1]:
# %pip install PyMuPDF

In [2]:
# %pip install tiktoken

In [3]:
# %pip install langchain_openai

In [4]:
import re
import os
import config
from glob import glob

from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import Document

from dotenv import load_dotenv
load_dotenv()

True

In [5]:
# 설정 정보 읽기
CHUNK_SIZE = config.chunk_size
CHUNK_OVERLAP = config.chunk_overlap

MODEL_NAME  = config.model_name
EMBEDDING_NAME = config.embedding_name

COLLECTION_NAME = config.collection_name
PERSIST_DIRECTORY = config.persist_directory

path_list = glob("pdf/*.pdf")

document_list = []

# load -> 전처리 -> split -> metadata 추가 
for path in path_list:
    # Document Load
    loader = PyMuPDFLoader(path)
    load_docs = loader.load()

    # 전처리 - 관련사료, 한자 제거
    full_text = [doc.page_content for doc in load_docs] 
    full_text = ''.join(full_text)
    full_text = re.sub(r"관련사료", "", full_text)
    full_text = re.sub(r"\([一-龥]+\)", "", full_text)

    # Split
    splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        model_name=MODEL_NAME,
        chunk_size=CHUNK_SIZE,
        chunk_overlap=CHUNK_OVERLAP,
    )
    # spiltter.documents([Document, Document, ...]) -> list[Document]
    docs = splitter.split_text(full_text) # 문자열 split -> list[str]

    # Metadata 생성
    title = "_".join(os.path.splitext(os.path.basename(path))[0].split('_')[:2])
    sub_title = os.path.splitext(os.path.basename(path))[0].split('_')[-1]
    metadata = {
        "title":title,
        "sub_title": sub_title,
        "full_text": full_text
    }

    # Document 생성
    for doc in docs: # docs: split된 소설 내용들. list[str]
        _doc = Document(metadata=metadata, page_content=doc)
        document_list.append(_doc)

print(len(document_list))

c:\Users\Playdata\Desktop\class\SKN06_mycode 공부용\project\my\Lib\site-packages\langchain_community\document_loaders\parsers\pdf.py:322: UserWarning: Warning: Empty content on page 3 of document pdf\사건_조선_한산도해전.pdf
  warnings.warn(
c:\Users\Playdata\Desktop\class\SKN06_mycode 공부용\project\my\Lib\site-packages\langchain_community\document_loaders\parsers\pdf.py:322: UserWarning: Warning: Empty content on page 3 of document pdf\인물_조선_이제마.pdf
  warnings.warn(


3923


In [6]:
print(document_list[0])

page_content='고구려 건국
하늘의 자손, 나라를 세우다
기원전 37년 ~
 개요
고구려 건국의 주도세력은 압록강 중상류 지역에 살고 있던 주민 집단과 부여계의 남하 집단과
의 융합으로 이루어졌다. 또한 하천과 계곡을 기준으로 만들어진 ‘나’라고 하는 작은 정치 단
위들의 상호 협력과 정복 과정을 통해 국가를 형성하였다.
 고구려의 건국 설화
『삼국사기』 등에 전하는 건국 설화에 따르면 고구려를 세운 사람은 부여에서 남하한
주몽이다. 주몽의 아버지는 천제의 아들인 해모수이고, 어머니는 물을 다스
리는 하백의 딸 유화라고 전한다. 유화는 부모의 허락 없이 해모수와 정을 통하여 쫓
겨났다가 부여의 왕인 금와왕에게 거두어졌다. 부여 왕궁에 머물게 된 유화의 몸에는 햇
빛이 따라 다니며 비추었는데 이 때문에 임신이 되었다. 유화는 얼마 후 사람이 아닌 알을 낳았
고, 그 알을 깨고 나온 것이 바로 주몽이다. 주몽은 어렸을 때부터 활을 잘 쏘고 재주가 뛰어나
부여의 왕자 대소 등에게 시기의 대상이 되었다. 이에 부여에서 용납되지 못함을 알고, 자
신을 따르는 동료들과 함께 남쪽으로 도망하였다.
주몽은 기원전 37년 졸본천에 도읍을 정하고 고구려를 건국하였다. 원래 이 지역에 자리
를 잡고 있던 비류국의 송양왕과 경쟁 관계를 형성하기도 하였으나 타고난 재주
와 권능으로 제압하였고, 이후 주변국들에 대한 정복 활동을 통해 나라의 기틀을 다졌다. 기원전
19년(동명성왕 19) 부여에 남겨 두고 왔던 부인과 함께 아들 유리가 찾아오자 태자로 임명
하였고 같은 해에 세상을 떠났다. 사후에는 동명성왕이라 일컬어지게 되었다.
 고구려의 종족적 계통
고구려 건국 설화의 내용은 상당 부분 부여의 건국 설화인 동명 설화를 차용한 것이다. 따라서
그 내용을 있는 그대로 역사적 사실이라 믿기는 어렵다. 하지만 한편으로는 전혀 상관이 없는 집
단이 부여의 건국 설화를 차용해 사용했다는 것도 생각하기 힘들다. 부여계 집단이 남하하여 고
구려라는 정치 집단을 형성하는 데 참여한 것은 사

In [7]:
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

In [8]:
len(document_list)

3923

In [9]:
import time
embedding_model = OpenAIEmbeddings(
    model=EMBEDDING_NAME
)
# 없으면 생성 / 연결 + document들 추가
vector_store = Chroma(
    embedding_function=embedding_model,
    collection_name=COLLECTION_NAME,
    persist_directory=PERSIST_DIRECTORY
)
batch_size = 500
for i, _ in enumerate(range(0,len(document_list), batch_size)):
    d = document_list[i*batch_size:(i+1)*batch_size]
    # print(len(d))
    vector_store.add_documents(d)
    time.sleep(5)


In [10]:
# batch_size = 500
# max_len = 4109
# for i in range(max_len):
#     print(i*batch_size, (i+1) * batch_size)

0 500
500 1000
1000 1500
1500 2000
2000 2500
2500 3000
3000 3500
3500 4000
4000 4500
4500 5000
5000 5500
5500 6000
6000 6500
6500 7000
7000 7500
7500 8000
8000 8500
8500 9000
9000 9500
9500 10000
10000 10500
10500 11000
11000 11500
11500 12000
12000 12500
12500 13000
13000 13500
13500 14000
14000 14500
14500 15000
15000 15500
15500 16000
16000 16500
16500 17000
17000 17500
17500 18000
18000 18500
18500 19000
19000 19500
19500 20000
20000 20500
20500 21000
21000 21500
21500 22000
22000 22500
22500 23000
23000 23500
23500 24000
24000 24500
24500 25000
25000 25500
25500 26000
26000 26500
26500 27000
27000 27500
27500 28000
28000 28500
28500 29000
29000 29500
29500 30000
30000 30500
30500 31000
31000 31500
31500 32000
32000 32500
32500 33000
33000 33500
33500 34000
34000 34500
34500 35000
35000 35500
35500 36000
36000 36500
36500 37000
37000 37500
37500 38000
38000 38500
38500 39000
39000 39500
39500 40000
40000 40500
40500 41000
41000 41500
41500 42000
42000 42500
42500 43000
43000 43500


In [12]:
embedding_model = OpenAIEmbeddings(
    model=EMBEDDING_NAME
)
# 없으면 생성 / 연결 + document들 추가
vector_store = Chroma.from_documents(
    documents=document_list,
    embedding=embedding_model,
    collection_name=COLLECTION_NAME,
    persist_directory=PERSIST_DIRECTORY
)
vector_store._collection.count()

RateLimitError: Error code: 429 - {'error': {'message': 'Request too large for text-embedding-3-large in organization org-hajyXttwwoW8jtQWO6T1Ijh3 on tokens per min (TPM): Limit 1000000, Requested 1083607. The input or output tokens must be reduced in order to run successfully. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}